In [6]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os

In [7]:
# Set the path to the dataset
dataset_path = r"D:\archive (3)\Driver Drowsiness Dataset (DDD)"

# Prepare data arrays
image_data = []
labels = []

# Define label mapping: 'drowsy' -> 0, 'awake' -> 1
label_map = {'Drowsy': 0, 'Awake': 1}

# Load images from the dataset
for label in label_map:
    label_dir = os.path.join(dataset_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))  # Resize to 64x64 for the CNN
        image_data.append(img)
        labels.append(label_map[label])

# Convert to numpy arrays and normalize images
image_data = np.array(image_data) / 255.0
labels = np.array(labels)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Two output classes: 'drowsy' and 'awake'
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

C:\Users\shash\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,625,410 (6.20 MB)

 Trainable params: 1,625,410 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Use data augmentation to improve generalization
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(X_train)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test, y_test), epochs=20)

Epoch 1/20


C:\Users\shash\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 71s 64ms/step - accuracy: 0.7789 - loss: 0.4203 - val_accuracy: 0.9812 - val_loss: 0.0472
Epoch 2/20
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 73s 70ms/step - accuracy: 0.9825 - loss: 0.0522 - val_accuracy: 0.9982 - val_loss: 0.0085
Epoch 3/20
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 72s 68ms/step - accuracy: 0.9882 - loss: 0.0333 - val_accuracy: 0.9952 - val_loss: 0.0160
Epoch 4/20
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 75s 72ms/step - accuracy: 0.9931 - loss: 0.0233 - val_accuracy: 0.9995 - val_loss: 0.0027
Epoch 5/20
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 71s 68ms/step - accuracy: 0.9937 - loss: 0.0195 - val_accuracy: 0.9992 - val_loss: 0.0025
Epoch 6/20
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 75s 71ms/step - accuracy: 0.9952 - loss: 0.0139 - val_accuracy: 0.9951 - val_loss: 0.0218
Epoch 7/20
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 71s 68ms/step - accuracy: 0.9944 - loss: 0.0181 - val_accuracy: 0.9980 - val_loss: 0.0077
Epoch 8/20
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 73s 70ms/step - accuracy: 0.9946 - loss: 0.01

In [11]:
model.save("drowsiness_detection_model.h5")